# Pörssisähkön hinta ja kulutus
Yhdistää Fingrid Datahubista manuaalisesti ladatun kulutuksen Entso-e Transparency Platform SFTP-palvelusta haettuun Nord Pool -pörssisähkön hintahistoriaan.

Ennen käyttöä:

* Käyttäjän tulee ladata itse tunneittainen kulutusdata osoitteesta https://oma.datahub.fi/.
* Käyttäjän tulee rekisteröityä Entso-e Transparency Platform -palveluun osoitteessa https://transparency.entsoe.eu/.

## Konfiguraatio

Entso-e tiedostojen sijainti määräytyy muuttujan `entso_e_data_local_folder` mukaan. Sijainti voi olla sama kuin kulutustietotiedostoilla muuttujan `consumption_data_local_folder` mukaan ja lopputuotteen hakmistolla muuttujan `output_data_local_folder`.

In [1]:
from getpass import getpass
from pathlib import Path
from glob import glob
import pandas as pd
import paramiko
import os
from datetime import datetime, timedelta, timezone

entso_e_data_local_folder = f"{Path.home()}/electricity_data" # User configurable
consumption_data_local_folder = f"{Path.home()}/electricity_data" # User configurable
output_data_local_folder = f"{Path.home()}/electricity_data" # User configurable
local_host_keys_filename = f"{Path.home()}/known_hosts" # User configurable

os.makedirs(entso_e_data_local_folder, exist_ok=True) # Create folder if does not exist

def get_entso_e_data_filename(year, month):
    return f"{year}_{month:02}_DayAheadPrices_12.1.D.csv"

## Kulutustietojen luku
Kaikki `*.csv` -tiedostot ladataan kulutustietokansiosta ja suodatetaan vain toteutuneita tuntikohtaisia kWh-kulutustietoja sisältäviksi. Hyväksyttyjen tiedostojen kulutusdatat yhdistetään.

In [9]:
consumption_data_datetime_col = "Alkuaika"
consumption_data_consumption_col = "Määrä"
consumption_data_resolution_col = "Resoluutio"
consumption_data_unit_col = "Yksikkötyyppi"
consumption_data_quality_col = "Laatu"

def is_consumption_data(filename):
    with open(filename, encoding="utf-8") as file:
        first_line = file.readline()
        return consumption_data_datetime_col in first_line and consumption_data_consumption_col in first_line

consumption_dict = {}
year_month_dict = {}

print(f"Reading consumption data from {consumption_data_local_folder}/*.csv:")
for filename in filter(is_consumption_data, glob(f"{consumption_data_local_folder}/*.csv")):
    df = pd.read_csv(filename, encoding="utf-8", sep=";", decimal=",", index_col=consumption_data_datetime_col)
    df = df[(df[consumption_data_quality_col] == "OK") & (df[consumption_data_resolution_col] == "PT1H") & (df[consumption_data_unit_col] == "kWh")]
    df.index = pd.to_datetime(df.index, utc=True)
    df[consumption_data_consumption_col] = df[consumption_data_consumption_col]
    num_hours = 0
    for index, row in df.iterrows():
        val = row[consumption_data_consumption_col]
        if index in consumption_dict:
            consumption_dict[index] = max(consumption_dict[index], val) # A larger consumption is probably more up ot date
        else:
            consumption_dict[index] = val
        year_month_dict[(index.year, index.month)] = True
        num_hours += 1
    print(f"Read {num_hours} hours from {filename}")

print()
print("Consumption data read for the following (year, month) combinations:")
print(sorted(year_month_dict.keys()))

Reading consumption data from C:\Users\o/electricity_data/*.csv:
Read 2195 hours from C:\Users\o/electricity_data\consumption (1).csv
Read 1584 hours from C:\Users\o/electricity_data\consumption 2018.csv
Read 8760 hours from C:\Users\o/electricity_data\consumption 2019.csv
Read 8784 hours from C:\Users\o/electricity_data\consumption 2020.csv
Read 8760 hours from C:\Users\o/electricity_data\consumption 2021.csv
Read 6691 hours from C:\Users\o/electricity_data\consumption 2022 aiemmin ladattu.csv
Read 7223 hours from C:\Users\o/electricity_data\consumption 2022.csv
Read 7223 hours from C:\Users\o/electricity_data\consumption.csv

Consumption data read for the following (year, month) combinations:
[(2018, 10), (2018, 11), (2018, 12), (2019, 1), (2019, 2), (2019, 3), (2019, 4), (2019, 5), (2019, 6), (2019, 7), (2019, 8), (2019, 9), (2019, 10), (2019, 11), (2019, 12), (2020, 1), (2020, 2), (2020, 3), (2020, 4), (2020, 5), (2020, 6), (2020, 7), (2020, 8), (2020, 9), (2020, 10), (2020, 11), (

## Entso-e kirjautumistietojen syöttö

In [3]:
user = input("Entso-e e-mail:")
password = getpass("Entso-e password:")
print("Entso-e SFTP username and password set")

Entso-e SFTP username and password set


## Hintahistorian lataus Entso-e SFTP-serveriltä

Hintahistoria ladataan kulutustietoja vastaavien kuukausien ajalta muuttujan `entso_e_data_local_folder` määräämään kansioon. Vain päiväyksiltään paikallisia uudemmat tiedostot ladataan Entso-e SFTP-serveriltä.

In [4]:
entso_e_host = "sftp-transparency.entsoe.eu"
entso_e_port = 22
entso_e_data_remote_folder = "/TP_export/DayAheadPrices_12.1.D"

class PromptPolicy(paramiko.client.MissingHostKeyPolicy):
    def missing_host_key(self, client, hostname, key):
        answer = input(f"Accept new key {key.get_base64()} (y/n)?")
        if answer == "y":
            answer = input(f"Save key in {local_host_keys_filename} (y/n)?")
            if answer == "y":
                client._host_keys.add(hostname, key.get_name(), key)
                Path(client._host_keys_filename).touch()
                client.save_host_keys(client._host_keys_filename)
                print(f"Saved key in {client._host_keys_filename}")
            return
        raise Exception("Unknown host key, not accepted by user")

ssh = paramiko.client.SSHClient()
ssh.load_system_host_keys()
try:
    ssh.load_host_keys(local_host_keys_filename)
except:
    None
promptPolicy = PromptPolicy()
ssh.set_missing_host_key_policy(promptPolicy)
try:
    print("Opening Entso-e SSH connection")
    ssh.connect(entso_e_host, port=entso_e_port, username=user, password=password)
    try:
        print("Opening Entso-e SFTP connection using the SSH connection")
        sftp = ssh.open_sftp()
        try:
            print(f"Downloading data to {entso_e_data_local_folder}:")
            for year, month in sorted(year_month_dict.keys()):                
                entso_e_data_filename = get_entso_e_data_filename(year, month)
                try:
                    st_mtime = sftp.stat(f"{entso_e_data_remote_folder}/{entso_e_data_filename}").st_mtime
                    try:
                        local_st_mtime = os.stat(f"{entso_e_data_local_folder}/{entso_e_data_filename}").st_mtime
                    except:
                        local_st_mtime = 0
                    if st_mtime == local_st_mtime:
                        print(f"{entso_e_data_filename} already up to date")
                    else:
                        print(f"{entso_e_data_filename} downloading...")
                        sftp.get(f"{entso_e_data_remote_folder}/{entso_e_data_filename}", f"{entso_e_data_local_folder}/{entso_e_data_filename}")
                        try:
                            os.utime(f"{entso_e_data_local_folder}/{entso_e_data_filename}", (st_mtime, st_mtime))
                        except:
                            print("ERROR: Could not update modified time")
                except:
                    print(f"{entso_e_data_filename} not available from Entso-e")
            print(f"Finished downloading data")
        except:
            print("ERROR downloading data")        
        sftp.close()
        print("Closed SFTP connection")
    except:
        print("ERROR opening SFTP connection")
    ssh.close()
    print("Closed SSH connection")
except:
    print("ERROR opening SSH connection")


Opening Entso-e SSH connection
Opening Entso-e SFTP connection using the SSH connection
2018_10_DayAheadPrices_12.1.D.csv already up to date
2018_11_DayAheadPrices_12.1.D.csv already up to date
2018_12_DayAheadPrices_12.1.D.csv already up to date
2019_01_DayAheadPrices_12.1.D.csv already up to date
2019_02_DayAheadPrices_12.1.D.csv already up to date
2019_03_DayAheadPrices_12.1.D.csv already up to date
2019_04_DayAheadPrices_12.1.D.csv already up to date
2019_05_DayAheadPrices_12.1.D.csv already up to date
2019_06_DayAheadPrices_12.1.D.csv already up to date
2019_07_DayAheadPrices_12.1.D.csv already up to date
2019_08_DayAheadPrices_12.1.D.csv already up to date
2019_09_DayAheadPrices_12.1.D.csv already up to date
2019_10_DayAheadPrices_12.1.D.csv already up to date
2019_11_DayAheadPrices_12.1.D.csv already up to date
2019_12_DayAheadPrices_12.1.D.csv already up to date
2020_01_DayAheadPrices_12.1.D.csv already up to date
2020_02_DayAheadPrices_12.1.D.csv already up to date
2020_03_Day

## Hintatietojen lataaminen

In [19]:
entso_e_data_datetime_col = "DateTime"
entso_e_data_area_col = "AreaCode"
entso_e_data_resolution_col = "ResolutionCode"
entso_e_data_currency_col = "Currency"
entso_e_data_price_col = "Price"

price_dict = {}

print(f"Reading price data...")
for year, month in sorted(year_month_dict.keys()):
    try:
        filename = get_entso_e_data_filename(year, month)
        df = pd.read_csv(f"{entso_e_data_local_folder}/{filename}", encoding="utf-8", sep="\t", decimal=".", index_col=entso_e_data_datetime_col)
        df = df[(df[entso_e_data_area_col] == "10YFI-1--------U") & (df[entso_e_data_resolution_col] == "PT60M") & (df[entso_e_data_currency_col] == "EUR")]
        df.index = pd.to_datetime(df.index, utc=True)
        df[entso_e_data_price_col] = df[entso_e_data_price_col]
        for index, row in df.iterrows():
            price_dict[index] = row[entso_e_data_price_col]
    except:
        print(f"ERROR reading/parsing {entso_e_data_local_folder}/{filename}")
        
print("Finished reading price data")

Reading price data...
Finished reading price data


## Hinta- ja kulutustietojen yhdistäminen

Anna tarkasteltavan aikavälin ensimmäinen tunti muuttujassa `start_datetime_string` ja viimeistä tuntia seuraava tunti muuttujassa `end_datetime_string` ISO-formaatissa. Ajat voivat määritellä myös aikavyöhykkeen, esim. `2011-11-04T00:00:00+02:00`. Jos aikavyöhykettä ei määritetä, käytetään tietokoneen aikavyöhykettä.

Yhdistetyt kulutus- ja hintatiedot tallentuvat aiemmin määritetyn muuttujan `output_data_local_folder` mukaiseen hakemistoon nimellä `Consumption and spot price ALKUAIKA to LOPPUAIKA.csv` johon sijoitetaan edellämainitut ajat. CSV-tiedostoon kirjoitetut ajat ovat ISO-formaatissa UTC-aikavyöhykkeessä.

In [18]:
start_datetime_string = "2022-11-01" # User configurable. Must be the start of an hour
end_datetime_string = "2022-12-01" # User configurable. Note that this datetime is excluded but the previous hour will be included

start_datetime = datetime.fromisoformat(start_datetime_string).astimezone(timezone.utc)
end_datetime = datetime.fromisoformat(end_datetime_string).astimezone(timezone.utc)
hour_timedelta = timedelta(hours=1)
t = start_datetime
datetimes = []
consumptions = []
prices = []
while t < end_datetime:
    datetimes.append(t)
    try:
        consumptions.append(consumption_dict[t])
    except:
        consumptions.append("")
    try:
        prices.append(price_dict[t])
    except:
        prices.append("")
    t += hour_timedelta
output = pd.DataFrame({"DateTime": datetimes, "Consumption (kWh)": consumptions, "Price (€/MWh)": prices})
output.set_index("DateTime")
output_filename = f"{output_data_local_folder}/Consumption and spot price {start_datetime_string.replace('+','p')} to {end_datetime_string.replace('+','p')}.csv"
output.to_csv(output_filename, index=False)
print(f"Wrote output file {output_filename}")

Wrote output file C:\Users\o/electricity_data/Consumption and spot price 2022-11-01 to 2022-12-01.csv
